In [90]:
import string
from collections import Counter

import requests as requests
import pandas as pd
from bs4 import BeautifulSoup

response = requests.get('https://minecraft.wiki/smelting')
print(response.status_code)
soup = BeautifulSoup(response.content, 'html.parser')

print(soup.p.get_text())

200
Smelting is the process of obtaining refined goods from raw materials by heating them in a furnace, blast furnace or smoker. When items are smelted in either type of furnace, experience is dropped. Like crafting, smelting uses recipes to determine what item is produced.



In [91]:
dfl = pd.read_html(response.content, header=0)
df = dfl[0]
dfstr = df.astype(str)
df

,Product,Ingredient,Exp,Usage
0,Baked Potato,Potato,0.35,"Fills 5 (), while raw fills 1 ()."
1,Dried Kelp,Kelp,0.10,Fills 1 () and can be eaten faster than other ...
2,Steak,Raw Beef,0.35,"Fills 8 (), while raw fills 3 ()."
3,Cooked Porkchop,Raw Porkchop,0.35,"Fills 8 (), while raw fills 3 ()."
4,Cooked Mutton,Raw Mutton,0.35,"Fills 6 (), while raw fills 2 ()."
5,Cooked Chicken,Raw Chicken,0.35,"Fills 6 (), while raw fills 2 () and has a 30%..."
6,Cooked Rabbit,Raw Rabbit,0.35,"Fills 5 (), while raw fills 3 ()."
7,Cooked Cod,Raw Cod,0.35,"Fills 5 (), while raw fills 2 ()."
8,Cooked Salmon,Raw Salmon,0.35,"Fills 6 (), while raw fills 2 ()."


In [92]:
words = (
    pd.Series(dfstr.values.flatten())
    .str.split()
    .explode()
    .value_counts()
    .reset_index()
)

words.columns = ['Word', 'Count']

words = words[words['Word'] != 'nan']
words


,Word,Count
0,Fills,9
1,while,9
2,raw,8
3,"(),",8
4,0.35,8
5,fills,8
6,().,7
7,Raw,7
8,Cooked,6
9,2,4


In [93]:
divs = soup.find('div', class_='mw-parser-output')
for edit_link in divs.find_all(class_='mw-editsection'):
    edit_link.decompose()

header_to_del = ['Navigation', 'References']

for header_name in header_to_del:
    header = divs.find(id=header_name)
    if header:
        header.decompose()

classes_to_del = ['mw-references-wrap']

for class_name in classes_to_del:
    classes = divs.find_all(class_=class_name)
    for clas in classes:
        clas.decompose()

for navbox in divs.find_all(class_='navbox'):
    navbox.decompose()

text = divs.get_text()

with open('smeltin_test.txt', 'w') as f:
    f.write(text)

# Some more classes to nuke found by llms
#     wrappers_to_nuke = [
#     'mw-references-wrap',
#     'navbox',
#     'infobox',
#     'toc',
#     'mw-editsection',
#     'hatnote',            # "See also:", "Main article:"
#     'searchaux',          # Klasa pomocnicza często występująca z hatnote/msgbox
#     'msgbox',             # Ramki z ostrzeżeniami/informacjami technicznymi
#     'thumb',              # Kontenery z obrazkami i podpisami (opcjonalnie)
#     'figure',             # Znacznik HTML5 dla obrazków (często na Wiki)
#     'catlinks',           # Kategorie na samym dole strony
#     'printfooter',        # "Retrieved from..." na dole
#     'mw-indicators'       # Ikonki w prawym górnym rogu (np. kłódka)
# ]



<bound method PageElement.get_text of <div class="mw-content-ltr mw-parser-output" dir="ltr" lang="en"><figure typeof="mw:File/Frame"><a class="mw-file-description" href="/w/File:Furnace_GUI.png"><img class="mw-file-element" data-file-height="332" data-file-width="352" decoding="async" height="332" loading="lazy" src="/images/Furnace_GUI.png?8d780" width="352"/></a><figcaption>The <a href="/w/Furnace" title="Furnace">furnace</a> interface. Note how the spaces are displayed.</figcaption></figure>
<p><b>Smelting</b> is the process of obtaining refined goods from raw materials by heating them in a <a href="/w/Furnace" title="Furnace">furnace</a>, <a class="mw-redirect" href="/w/Blast_furnace" title="Blast furnace">blast furnace</a> or <a href="/w/Smoker" title="Smoker">smoker</a>. When items are smelted in either type of furnace, <a href="/w/Experience" title="Experience">experience</a> is dropped. Like <a href="/w/Crafting" title="Crafting">crafting</a>, smelting uses recipes to determin

In [94]:
text = text.lower()

to_del_signs = string.punctuation + string.digits  + '×' + '–' + '⁄' + '\u200c'

for sign in to_del_signs:
    text = text.replace(sign, ' ')

words_list = text.split()

words_list

['the',
 'furnace',
 'interface',
 'note',
 'how',
 'the',
 'spaces',
 'are',
 'displayed',
 'smelting',
 'is',
 'the',
 'process',
 'of',
 'obtaining',
 'refined',
 'goods',
 'from',
 'raw',
 'materials',
 'by',
 'heating',
 'them',
 'in',
 'a',
 'furnace',
 'blast',
 'furnace',
 'or',
 'smoker',
 'when',
 'items',
 'are',
 'smelted',
 'in',
 'either',
 'type',
 'of',
 'furnace',
 'experience',
 'is',
 'dropped',
 'like',
 'crafting',
 'smelting',
 'uses',
 'recipes',
 'to',
 'determine',
 'what',
 'item',
 'is',
 'produced',
 'contents',
 'methods',
 'furnace',
 'blast',
 'furnace',
 'and',
 'smoker',
 'recipes',
 'food',
 'ores',
 'gear',
 'furnace',
 'only',
 'fuel',
 'hopper',
 'automation',
 'achievements',
 'advancements',
 'history',
 'java',
 'edition',
 'bedrock',
 'edition',
 'new',
 'nintendo',
 'ds',
 'edition',
 'issues',
 'trivia',
 'gallery',
 'screenshots',
 'see',
 'also',
 'references',
 'navigation',
 'methods',
 'furnace',
 'main',
 'article',
 'furnace',
 'the',
 

In [95]:
counter = Counter(words_list)
counter


Counter({'the': 159,
         'used': 121,
         'and': 98,
         'a': 90,
         'be': 77,
         'fuel': 76,
         'ticks': 73,
         'can': 72,
         'furnace': 66,
         'to': 65,
         'as': 61,
         'of': 56,
         'is': 50,
         'in': 47,
         'now': 43,
         'item': 39,
         'none': 36,
         'smelting': 33,
         'items': 33,
         'for': 30,
         'or': 28,
         'slot': 25,
         'edition': 24,
         'which': 24,
         'iron': 23,
         'block': 23,
         'smelted': 21,
         'only': 21,
         'wooden': 21,
         'w': 21,
         'raw': 20,
         'output': 20,
         'when': 19,
         'from': 18,
         'an': 18,
         'if': 18,
         'input': 18,
         'note': 17,
         'experience': 17,
         'fills': 17,
         'blocks': 17,
         'copper': 17,
         'with': 17,
         'also': 16,
         'smelt': 16,
         'added': 16,
         'furnaces': 16,
  

In [111]:
import json, os

if os.path.exists('word_counts.json'):
    with open('word_counts.json', 'r', encoding='utf-8') as f:
        old_json = json.load(f)
        old_counter = Counter(old_json)
else:
    old_counter = Counter()

old_counter.update(counter)

with open('word_counts.json', 'w', encoding='utf-8') as f:
    json.dump(counter, f, ensure_ascii=False, indent=4)

